In [4]:
!pip install workadays
!pip install sidrapy

In [5]:
import math
import sidrapy
from bcb import sgs
import pandas as pd
import datetime as dt
from datetime import datetime
from workadays import workdays as wd
from datetime import date

In [9]:
def truncate(num, n):
    integer = int(num * (10**n))/(10**n)
    return float(integer)

In [7]:
hoje = dt.date(2023,8,3)

In [28]:
# PRECIFICANDO LTN

principal = 1000
vencimento = dt.date(2026,7,1)
i = 11.3980

dias = wd.networkdays(hoje,vencimento,252)/252

PU = truncate(principal/(1+i/100)**(dias),6)

print(PU)

714.452233


In [123]:
# PRECIFICANDO NTN-F

principal = 1000
cupom = 0.1
vencimento = dt.date(2033,1,1)
i = 11.8329

taxa_cupom = round((((1+cupom)**0.5)-1),8)

if hoje.month < 7:
    fluxo_1 = '01/07/' + str(hoje.year)
else:
    fluxo_1 = '01/01/'+ (str(hoje.year + 1))

fluxo_total = []
fluxo_total.append(fluxo_1)
fluxo_1 = datetime.strptime(fluxo_1,'%d/%m/%Y').date()

while(fluxo_1 < vencimento):
    fluxo_n = '01/' + (str(fluxo_1.month + 6)) + '/' + str(fluxo_1.year)
    if float(str(fluxo_1.month + 6)) <= 7:
        fluxo_n = '01/07/' + str(fluxo_1.year)
    else:
        fluxo_n = '01/01/'+ (str(fluxo_1.year + 1))
    fluxo_total.append(fluxo_n)
    fluxo_1 = datetime.strptime(fluxo_n,'%d/%m/%Y').date()

fluxo_pgto = []
for data in fluxo_total:
    data = datetime.strptime(data,'%d/%m/%Y').date()
    data = wd.workdays(wd.workdays(data,-1),+1)
    fluxo_pgto.append(data)

diaano_pgto = []
for data in fluxo_pgto:
    data = round((wd.networkdays(hoje,data,252))/252,16)
    diaano_pgto.append(data)
    
pg_cupom = []
for data in diaano_pgto:
    valor_cupom = round((taxa_cupom*principal)/(1+i/100)**data,8)
    pg_cupom.append(valor_cupom)
    
valor_principal = round(principal/(1+i/100)**(diaano_pgto[-1]),8)

val_cupom = round(pd.DataFrame(pg_cupom).sum(),8)

PU = truncate(val_cupom + valor_principal,6)

PU

938.118375

In [12]:
# VNA IPCA
ipca_proj = 0.26/100
ipca_raw = sidrapy.get_table(table_code = '1737',
                             territorial_level = '1',
                             ibge_territorial_code = 'all',
                             variable = '63,2266',
                             period = 'last%20472')
ipca = ipca_raw.loc[1:,['V', 'D2C', 'D3N']].rename(columns = {'V': 'value','D2C': 'date','D3N': 'variable'})
ipca_indice = ipca.loc[(ipca['date'] > '200005') & (ipca['variable'].str.startswith('IPCA - Número-índice')),
                       ['date', 'variable','value']]

ipca_var = ipca.loc[(ipca['date'] > '200005') & (ipca['variable'].str.startswith('IPCA - Variação mensal')),
                       ['date', 'variable', 'value']]

if str(hoje.day) == '15':
    if len(str(hoje.month-1)) < 2:
        vna = float(ipca_indice[ipca_indice['date'] == str(hoje.year)+ '0' + str(hoje.month-1)]['value'])/float(ipca_indice[(ipca_indice['date'] == '200006')]['value'])
    else:
        vna = float(ipca_indice[ipca_indice['date'] == str(hoje.year) + str(hoje.month-1)]['value'])/float(ipca_indice[(ipca_indice['date'] == '200006')]['value'])
        
    vna_ipca = vna*100
                                                                                                           
if str(hoje.day) < '15':
    if len(str(hoje.month-1)) < 2:
        mes_passado = '15' + '/' + '0' + str(hoje.month-1) + '/' + str(hoje.year)
    else:
        mes_passado = datetime.strptime(mes_passado,'%d/%m/%Y').date()
        
    mes_passado = datetime.strptime(mes_passado,'%d/%m/%Y').date()
        
    if len(str(hoje.month)) < 2:
        mes_atual = '15' + '/' + '0' + str(hoje.month) + '/' + str(hoje.year)
    else:
        mes_atual = '15' + '/' + str(hoje.month) + '/' + str(hoje.year)
               
    mes_atual = datetime.strptime(mes_atual,'%d/%m/%Y').date()
    
    if len(str(hoje.month-2)) < 2:
        mes_proximo = '15' + '/' + '0' + str(hoje.month-2) + '/' + str(hoje.year)
    else:
        mes_proximo = '15' + '/' + str(hoje.month-2) + '/' + str(hoje.year)
        
    mes_proximo = datetime.strptime(mes_proximo,'%d/%m/%Y').date()
        
    du1 = wd.networkdays(mes_passado,mes_atual,252)
    du2 = wd.networkdays(mes_atual,mes_proximo,252)
    
    vna_antes = float(ipca_indice[ipca_indice['date'] == str(hoje.year)+ '0' + str(hoje.month-2)]['value'])/float(ipca_indice[(ipca_indice['date'] == '200006')]['value'])
    ipca_prorata = float(ipca_var[ipca_var['date'] == str(hoje.year)+ '0' + str(hoje.month-2)]['value'])
    
    vna_ipca = truncate(vna_antes*ipca_prorata**(du1/du2)*1000,6)

if str(hoje.day) > '15':
    if len(str(hoje.month-1)) < 2:
        mes_passado = '15' + '/' + '0' + str(hoje.month-1) + '/' + str(hoje.year)
    else:
        mes_passado = '15' + '/' + str(hoje.month-1) + '/' + str(hoje.year)
    mes_passado = datetime.strptime(mes_passado,'%d/%m/%Y').date()
        
    if len(str(hoje.month)) < 2:
        mes_atual = '15' + '/' + '0' + str(hoje.month) + '/' + str(hoje.year)
    else:
        mes_atual = '15' + '/' + str(hoje.month) + '/' + str(hoje.year)
    mes_atual = datetime.strptime(mes_atual,'%d/%m/%Y').date()
    
    if len(str(hoje.month+1)) < 2:
        mes_proximo = '15' + '/' + '0' + str(hoje.month+1) + '/' + str(hoje.year)
    else:
        mes_passado = '15' + '/' + str(hoje.month+1) + '/' + str(hoje.year)
    mes_proximo = datetime.strptime(mes_proximo,'%d/%m/%Y').date()
        
    du1 = round(abs(wd.networkdays(mes_atual,hoje,252)),12)
    du2 = round(wd.networkdays(mes_atual,mes_proximo,252),12)
    
    vna_antes = truncate(float(ipca_indice[ipca_indice['date'] == str(hoje.year)+ '0' + str(hoje.month-1)]['value'])/float(ipca_indice[(ipca_indice['date'] == '200006')]['value']),9)  
    vna_ipca = truncate(vna_antes*(1+ipca_proj)**(du1/du2)*1000,6)    
    
vna_ipca

4135.580365

In [10]:
# PRECIFICANDO NTN-B

cupom = 0.06
vencimento = dt.date(2026,8,15)
i = 5.0874
vna = 4126.821021

taxa_cupom = round((((1+cupom)**0.5)-1),8)

if vencimento.year%2 == 0:
    ano_vencimento ='par'
else:
    ano_vencimento ='impar'

if ano_vencimento == 'par':
    if hoje.month < 2:
        fluxo_1_par = '15/02/' + str(hoje.year)
    if hoje.month == 2 and hoje.day < 15:
        fluxo_1_par = '15/02/' + str(hoje.year)
    if hoje.month == 2 and hoje.day >= 15:
        fluxo_1_par = '15/08/' + str(hoje.year)
    if hoje.month > 2 and hoje.month < 8:
        fluxo_1_par = '15/08/' + str(hoje.year)
    if hoje.month == 8 and hoje.day < 15:
        fluxo_1_par = '15/08/' + str(hoje.year)
    if hoje.month == 8 and hoje.day >= 15:
        fluxo_1_par = '15/02/' + (str(hoje.year + 1))
    if hoje.month > 8:
        fluxo_1_par = '15/02/' + (str(hoje.year + 1))  
        
    fluxo_total = []
    fluxo_total.append(fluxo_1_par)
    fluxo_1_par = datetime.strptime(fluxo_1_par,'%d/%m/%Y').date()
        
    while(fluxo_1_par < vencimento):
        fluxo_n_par = '15/' + (str(fluxo_1_par.month + 6)) + '/' + str(fluxo_1_par.year)
        if float(str(fluxo_1_par.month + 6)) > 8:
            fluxo_n_par = '15/02/' + (str(fluxo_1_par.year + 1))
        else:
            fluxo_n_par = '15/08/' + str(fluxo_1_par.year)
            
        fluxo_total.append(fluxo_n_par)
        fluxo_1_par = datetime.strptime(fluxo_n_par,'%d/%m/%Y').date()

if ano_vencimento == 'impar':
    if hoje.month < 5:
        fluxo_2 = '15/05/' + str(hoje.year)
    if hoje.month == 5 and hoje.day < 15:
        fluxo_2 = '15/05/' + str(hoje.year)
    if hoje.month == 5 and hoje.day >= 15:
        fluxo_2 = '15/11/' + str(hoje.year)
    if hoje.month > 5 and hoje.month < 11:
        fluxo_2 = '15/11/' + str(hoje.year)
    if hoje.month == 11 and hoje.day < 15:
        fluxo_2 = '15/11/' + str(hoje.year)
    if hoje.month == 8 and hoje.day >= 15:
        fluxo_2 = '15/05/' + (str(hoje.year + 1))
    if hoje.month > 8:
        fluxo_2 = '15/05/' + (str(hoje.year + 1))
        
    fluxo_total = []
    fluxo_total.append(fluxo_2)
    fluxo_2 = datetime.strptime(fluxo_2,'%d/%m/%Y').date()
        
    while(fluxo_2 < vencimento):
        fluxo_n = '15/' + (str(fluxo_2.month + 6)) + '/' + str(fluxo_2.year)
        if float(str(fluxo_2.month + 6)) > 11:
            fluxo_n = '15/05/' + (str(fluxo_2.year + 1))
        else:
            fluxo_n = '15/11/' + str(fluxo_2.year)
            
        fluxo_total.append(fluxo_n)
        fluxo_2 = datetime.strptime(fluxo_n,'%d/%m/%Y').date()
            
fluxo_pgto = []
for data in fluxo_total:
    data = datetime.strptime(data,'%d/%m/%Y').date()
    data = wd.workdays(wd.workdays(data,-1),+1)
    fluxo_pgto.append(data)
                
diaano_pgto = []
for data in fluxo_pgto:
    data = truncate((wd.networkdays(hoje,data,252))/252,14)
    diaano_pgto.append(data)
                
pg_cupom = []
for data in diaano_pgto:
    valor_cupom = (taxa_cupom*100)/(1+i/100)**data
    pg_cupom.append(valor_cupom)
        
valor_principal = 100/(1+i/100)**(diaano_pgto[-1])
cotacao = truncate(pd.DataFrame(pg_cupom).sum() + valor_principal,4)/100
PU = truncate(cotacao*vna_ipca,6)
            
PU

4342.897242

In [21]:
# PRECIFICANDO LFT

taxa = 0.1923
vencimento = dt.date(2029,3,1)
dias = wd.networkdays(hoje,vencimento,252)/252

# VNA SELIC

selic_inicio = sgs.get({'selic':11}, start = '2000-07-01', end = hoje).astype('float')

selic_inicio['acumulada'] = 1000
for i in range(1,len(selic_inicio['selic'])):
    acumulada = selic_inicio['acumulada'].iloc[i-1] * (1+selic_inicio['selic'].iloc[i]/100)
    selic_inicio['acumulada'][i] = acumulada
    
vna_selic = truncate(selic_inicio['acumulada'].filter(like = str(wd.workdays(date.today(),-2)), axis = 0),6)

cotacao = truncate((1/(1+taxa/100)**dias),6)
PU = truncate(vna_selic*cotacao,6)
PU

C:\Users\fgomes\AppData\Local\Temp\ipykernel_10132\702838232.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selic_inicio['acumulada'][i] = acumulada


13114.098202

In [25]:
mes_passado

datetime.date(2023, 5, 15)

In [9]:
du1 = wd.networkdays(mes_passado,mes_atual,252)

AttributeError: 'str' object has no attribute 'year'

In [18]:
hoje.month-1

5

In [10]:
mes_passado

'15/05/2023'

In [11]:
mes_atual

datetime.date(2023, 6, 15)

In [6]:
ipca_indice.tail()

,date,variable,value
936,202302,IPCA - Número-índice (base: dezembro de 1993 =...,6563.0700000000000
938,202303,IPCA - Número-índice (base: dezembro de 1993 =...,6609.6700000000000
940,202304,IPCA - Número-índice (base: dezembro de 1993 =...,6649.9900000000000
942,202305,IPCA - Número-índice (base: dezembro de 1993 =...,6665.2800000000000
944,202306,IPCA - Número-índice (base: dezembro de 1993 =...,6659.9500000000000
